In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from datasist.structdata import detect_outliers

## Define the Dataset

In [ ]:
df = pd.read_csv("Bank Data.csv")
df.head()

## Exploratory Data Analysis

### Structure of the Data

In [ ]:
df.info()

Many numerical columns are considered as object:

Age must be an integer.
Annual_Income -> float.
Num_of_Loan -> integer.
Num_of_Delayed_Payment -> float64.
Changed_Credit_Limit -> float64.
Outstanding_Debt -> float64.
Credit_History_Age -> month counts (int).
Amount_invested_monthly -> float64.
Monthly_Balance -> float64.

### Summary Statistics

In [ ]:
df.describe()

### Duplicated Values

In [ ]:
df[df.duplicated()]

No duplicated values, it might be due to the unique columns.
We will check again once we apply further cleaning.

### Unique Values

In [ ]:
df.nunique()

All entries of ID and Credit_Utilization_Ratio are unique!
We have to drop these two columns.

In [ ]:
# why Payment_of_Min_Amount col has only 3 unique values? 
df['Payment_of_Min_Amount'].unique()

### Missing Values

In [ ]:
df.isnull().sum()

We may:
Drop columns with more than 20% missing values.
Impute missing values in numerical columns with mean, median, or mode.
Impute missing values  in categorical columns with mode.
Immutate using domain knowledge.

### Explore some Columns

There may be a relation between SSN and Customer_ID

In [ ]:
df['Customer_ID'].value_counts()

In [ ]:
df['SSN'].value_counts()

We got unexpected output where SSN value "#F%$D@*&8" which considered as corrupted data.
This could happen due to data entry errors.
Now we know that we have to handle it in the preprocessing phase.

In [ ]:
df['SSN'] = df['SSN'].replace('#F%$D@*&8', np.nan)

In [ ]:
df[['Customer_ID', 'SSN']] 

In [ ]:
df.groupby('Customer_ID')['SSN'].apply(set)

In [ ]:
for x in df.groupby('Customer_ID')['SSN'].apply(set):
    print(x)

As predicted, each Customer_ID is associated with one SSN.
We need to fill NaN values in the SSN column with the correct SSN, which can be obtained from the Customer_ID.

But, since both columns represent the same information, and the SSN is sensitive data,
besides that it requires preprocessing, we will drop the SSN column.

### Data Visualization

#### Month column

In [ ]:
px.histogram( df, 'Month')

####  Num_Bank_Accounts and Num_Credit_Card columns

There might be a relationship between Num_Bank_Accounts and Num_Credit_Card

In [ ]:
ff.create_distplot([df['Num_Bank_Accounts'], df['Num_Credit_Card']], ['Num_Bank_Accounts', 'Num_Credit_Card'], show_hist=False, show_rug=False)

They are almost the same, so we may drop one of them.

In [ ]:
# Reduce the range for a closer overview.
c1 = df[df['Num_Bank_Accounts'] < 100]['Num_Bank_Accounts']
c2 = df[df['Num_Credit_Card'] < 100]['Num_Credit_Card']
ff.create_distplot([c1, c2], ['Num_Bank_Accounts', 'Num_Credit_Card'], show_hist=False, show_rug=False)

In [ ]:
# Focus on the high-density part
c1 = df[df['Num_Bank_Accounts'] < 20]['Num_Bank_Accounts']
c2 = df[df['Num_Credit_Card'] < 20]['Num_Credit_Card']
ff.create_distplot([c1, c2], ['Num_Bank_Accounts', 'Num_Credit_Card'], show_hist=False, show_rug=False)

Even if they don't have the same numbers, it is clear that there is a meaningful relationship between them.

In [ ]:
df['Num_Bank_Accounts'].nunique()

In [ ]:
df['Num_Credit_Card'].nunique()

Num_Credit_Card has more unique values.

In [ ]:
# analyze statistics to decide which one to drop
df.describe()[['Num_Credit_Card', 'Num_Bank_Accounts']]

Two columns, 'Num_Credit_Card' and 'Num_Bank_Accounts', have similar distributions.
'Num_Credit_Card' has 819 unique values, while 'Num_Bank_Accounts' has 540 unique values.
To simplify the dataset and avoid handling higher cardinality, we will drop the 'Num_Credit_Card' column due to its high variability and potential outliers.
Further preprocessing will be performed on the 'Num_Bank_Accounts' column.

## Data Preprocessing

### Drop usless columns

In [ ]:
df.columns

In [ ]:
df.drop(['ID', 'Credit_Utilization_Ratio', 'SSN', 'Name', 'Num_Credit_Card'], axis=1, inplace=True)

In [ ]:
# Check for duplicates after removing the unique indices
df.duplicated().sum()

In [ ]:
# remaining columns
df.columns

### Feature Engineering & Transformation

#### Correct Data-types of Numerical Values

In [ ]:
def isNum(x):
    try:
        float(x)
        return True
    except:
        return False

##### Age to be integer.

In [ ]:
df['Age']

In [ ]:
temp = df[ df['Age'].apply(isNum) == False ]
temp['Age']

We notice that all non int values are trailed by "_"

In [ ]:
temp['Age'] = temp['Age'].apply(lambda x: x.strip('_'))

In [ ]:
temp[ temp['Age'].apply(isNum) == False ].head(50)

Perfect! Now we know that all the problem was in the trailing underscore.

In [ ]:
df['Age'] = df['Age'].str.strip('_').astype(int)
df['Age']

##### Annual_Income to be float.

In [ ]:
df['Annual_Income']

In [ ]:
df[df['Annual_Income'].apply(isNum) == False]['Annual_Income']

In [ ]:
# same problem here
df['Annual_Income'] = df['Annual_Income'].str.strip('_').astype(float)

In [ ]:
df[df['Annual_Income'].apply(isNum) == False]

##### Num_of_Loan to be integer.

In [ ]:
df['Num_of_Loan']

In [ ]:
df['Num_of_Loan'] = df['Num_of_Loan'].str.strip('_').astype(int)
df['Num_of_Loan']

##### Num_of_Delayed_Payment to be integer.

In [ ]:
df['Num_of_Delayed_Payment']

In [ ]:
# here, we have Nan values...
df['Num_of_Delayed_Payment'].isnull().sum()

In [ ]:
# use float type because of NaN values
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.strip('_').astype(float)
df['Num_of_Delayed_Payment']

##### Changed_Credit_Limit to be float64

In [ ]:
df['Changed_Credit_Limit']

In [ ]:
df[df['Changed_Credit_Limit'].apply(isNum) == False]['Changed_Credit_Limit']

In [ ]:
df[df['Changed_Credit_Limit'] == '_']['Changed_Credit_Limit']

In [ ]:
# all 1059 non numeric values are -, replace it by NaN using to_numeric function
df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')

In [ ]:
df[df['Changed_Credit_Limit'].apply(isNum) == False]

##### Outstanding_Debt to be float64

In [ ]:
df['Outstanding_Debt']

In [ ]:
df[df['Outstanding_Debt'].apply(isNum) == False]['Outstanding_Debt']

In [ ]:
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.strip('_').astype(float)

In [ ]:
df[df['Outstanding_Debt'].apply(isNum) == False]

##### Feature Extraction on Credit_History_Age

In [ ]:
df['Credit_History_Age']

In [ ]:
years = int(df['Credit_History_Age'][0].split(' ')[0])
years

In [ ]:
months = int(df['Credit_History_Age'][0].split(' ')[3])
months

In [ ]:
totMonths = years*12 + months
totMonths

In [ ]:
def convertToMonths(age):
    try:
        years = int(age.split(' ')[0])
        months = int(age.split(' ')[3])
        return years*12 + months
    except:
        return np.nan

In [ ]:
df['Credit_History_Age'].isnull().sum()

In [ ]:
df['Credit_History_Age(months)'] = df['Credit_History_Age'].apply(convertToMonths)
df['Credit_History_Age(months)']

In [ ]:
df['Credit_History_Age(months)'].isnull().sum()

Number of null values didn't change.

In [ ]:
df.drop('Credit_History_Age', axis=1, inplace=True)

Extract years from months

In [ ]:
df['Credit_History_Age(years)'] = df['Credit_History_Age(months)'].apply(lambda x: x/12)
df['Credit_History_Age(years)']

In [ ]:
df['Credit_History_Age(years)'].isnull().sum()

##### Amount_invested_monthly to be float64

In [ ]:
df['Amount_invested_monthly']

In [ ]:
df[ df['Amount_invested_monthly'].apply(isNum) == False]['Amount_invested_monthly']

So, we need to strip the underscore.

In [ ]:
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].str.strip('_').astype(float)
df['Amount_invested_monthly']

In [ ]:
df['Amount_invested_monthly'].isnull().sum()

##### Monthly_Balance to be float64

In [ ]:
df['Monthly_Balance']

In [ ]:
df['Monthly_Balance'].isnull().sum()

In [ ]:
df[ df['Monthly_Balance'].apply(isNum) == False]['Monthly_Balance']

In [ ]:
df[ df['Monthly_Balance'].apply(isNum) == False].shape[0]

In [ ]:
df[ df['Monthly_Balance'] == '__-333333333333333333333333333__'].shape[0]

So all the non-float values are garbage. Only 6 rows, so replace them by Nan, and impute it later.

In [ ]:
df['Monthly_Balance'].replace('__-333333333333333333333333333__', np.nan, inplace=True)

In [ ]:
df['Monthly_Balance'] = df['Monthly_Balance'].astype(float)
df['Monthly_Balance']

We had 562 null values with these 6, now we must have 568.

In [ ]:
df['Monthly_Balance'].isnull().sum()

done perfectly! ;)

##### Check for all datatypes

In [ ]:
df.info()

Age to be integer.                        DONE
Annual_Income to be float.                DONE
Num_of_Loan integer.                      DONE
Num_of_Delayed_Payment float64.           DONE
Changed_Credit_Limit float64              DONE
Outstanding_Debt float64                  DONE
Credit_History_Age Num of months          DONE 
Amount_invested_monthly float64           DONE
Monthly_Balance float64                   DONE

#### Categorical Values

In [ ]:
categoricalCols = df.select_dtypes(include=['object'])
for col in categoricalCols:
    print(col)
    print(df[col].unique())
    print()

##### Customer_ID

In [ ]:
from re import match
pattern = r'^CUS_0x[0-9a-fA-F]+$'
df['Customer_ID'].apply(lambda x: bool(match(pattern, x))).all()

No corrupted data in Customer_ID column.

In [ ]:
df['Customer_ID'].value_counts()

Why all customers appeared more than once?

In [ ]:
for id in df.sample(5)['Customer_ID']:
    print('Customer_ID: ', id)
    display(df[ df['Customer_ID'] == id])

We observe that the columns likely to vary between rows are primarily the 'Month' and other columns such as 'Monthly_Inhand_Salary'.

##### Month

In [ ]:
df['Month'].unique()

No data entry problems.

In [ ]:
df['Month'].value_counts()

It is normally distributed, but it is important because other data like Monthly_Inhand_Salary are depending on it.

##### Occupation

In [ ]:
df['Occupation'].value_counts()

In [ ]:
df['Occupation'] = df['Occupation'].replace('_______', np.nan)

In [ ]:
df['Occupation'].isnull().sum()

##### Type_of_Loan

In [ ]:
df['Type_of_Loan']

In [ ]:
df['Type_of_Loan'].isnull().sum()

In [ ]:
for x in df['Type_of_Loan'].unique():
    print(x)
    print()

Let's try to extract all Loan Types, if they are not many, then we will apply feature extraction on Type_of_Loan column.

In [ ]:
# Concatenate all 'Type_of_Loan' values into a single string
loanTypes = ', '.join(df['Type_of_Loan'].dropna())

# Split the string by comma to create a list of loan types
loanTypes = loanTypes.split(', ')

# Convert the list to a set to ensure all loan types are unique
loanTypes = set(loanTypes)
loanTypes

In [ ]:
# Remove the "and" from each element
loanTypes = {x.replace('and ', '') for x in loanTypes}
loanTypes

In [ ]:
# Feature Extraction
# extract loan types from 'Type_of_Loan'
for loan_type in loanTypes:
    # from the previous list, we get the names of the columns:
    col = f'Has_{loan_type.replace(" ", "_").replace("-", "_")}'
    # count the occurrences of the current column in the Type_of_Loan column
    # add the count to the current column:
    df[col] = df['Type_of_Loan'].apply(lambda x: str(x).count(loan_type) if isinstance(x, str) else float('nan'))

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

all have same number of nulls same as the parent column.

In [ ]:
# Let's compare Type_of_Loan column:
df['Type_of_Loan']

In [ ]:
# with the extracted columns:
df.iloc[:, -9:]

In [ ]:
# take a look of random samples:
df.sample(5)

In [ ]:
df['Num_of_Loan'] == df.iloc[:, -9:].sum(axis=1)

In [ ]:
df[ df['Num_of_Loan'] != df.iloc[:, -9:].sum(axis=1) ]

The are many rows, but it looks like there is an error in the data entry. So we will check again after fixing them.

##### Credit_Mix

In [ ]:
df['Credit_Mix'].unique()

In [ ]:
df['Credit_Mix'] = df['Credit_Mix'].replace('_', np.nan)

In [ ]:
df['Credit_Mix'].unique()

##### Payment_Behaviour

In [ ]:
df['Payment_Behaviour'].unique()

In [ ]:
df['Payment_Behaviour'] = df['Payment_Behaviour'].replace('!@9#%8', np.nan)
df['Payment_Behaviour'].unique()

### Data Cleaning

In [ ]:
df.describe()

#### Age

In [ ]:
df[df['Age'] == -500]['Age'].value_counts()

In [ ]:
df[df['Age'] < 0]['Age'].value_counts()

In [ ]:
# so all -ve values are -500. even 500 is not real value.
df['Age'] = df['Age'].replace(-500, np.nan)

In [ ]:
df[df['Age'] < 0].shape[0]

In [ ]:
df[df['Age'] > 100]['Age'].shape[0]

In [ ]:
# replace them with nulls
df.loc[df['Age'] > 100, 'Age'] = np.nan

In [ ]:
df['Age'].describe()

Assume the bank accepts 14 yo.

In [ ]:
px.box(df, 'Age')

#### Num_Bank_Accounts

In [ ]:
df[df['Num_Bank_Accounts'] == -1].shape[0]

In [ ]:
# it might be an error in data entry, and might mean to be 1 instead.
df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].replace(-1, 1)

In [ ]:
df['Num_Bank_Accounts'].describe()

In [ ]:
# assume max number of accounts is 20
df[df['Num_Bank_Accounts'] > 20].shape[0]

In [ ]:
df[df['Num_Bank_Accounts'] > 20] = np.nan

In [ ]:
px.box(df, 'Num_Bank_Accounts')

#### Num_of_Loan

As we said before. There are many values that are not accepted. Initially, let's see the range of the complete data.

In [ ]:
df[ df['Num_of_Loan'] == df.iloc[:, -9:].sum(axis=1) ]['Num_of_Loan'].describe()

so the normal value is between 0 and 9

In [ ]:
df[df['Num_of_Loan'] == 10]

In [ ]:
df[ df['Num_of_Loan'] != df.iloc[:, -9:].sum(axis=1) ]['Num_of_Loan'].unique()

All in-matched data does not correspond to reality.
It is clear that the discrepancies between columns are due to data entry errors in Num_of_Loan column.
We will derive the correct data from the other columns.

In [ ]:
df['Num_of_Loan'] = df.iloc[:, -9:].sum(axis=1)

In [ ]:
df['Num_of_Loan'].describe()

In [ ]:
df.isnull().sum()

In [ ]:
# Num_of_Loan must have same number of nulls as Type_of_Loan
df.loc[df['Type_of_Loan'].isnull(), 'Num_of_Loan'] = np.nan

In [ ]:
df['Num_of_Loan'].isnull().sum()

Now, Num_of_Loan, Type_of_Loan, and each type of loan, all have the same number of nulls.

In [ ]:
px.box(df, 'Num_of_Loan')

#### Delay_from_due_date

In [ ]:
df[df['Delay_from_due_date'] == -5].shape[0]

In [ ]:
df[df['Delay_from_due_date']<0].shape[0]

In [ ]:
df[df['Delay_from_due_date']<0]['Delay_from_due_date'].value_counts()

In [ ]:
# it might be an error in data entry.
df['Delay_from_due_date'] = df['Delay_from_due_date'].abs()

In [ ]:
df[df['Delay_from_due_date'] < 0].shape[0]

#### Num_of_Delayed_Payment

In [ ]:
df[df['Num_of_Delayed_Payment'] == -3].shape[0]

In [ ]:
df[df['Num_of_Delayed_Payment'] < 0].shape[0]

In [ ]:
df[df['Num_of_Delayed_Payment']<0]['Num_of_Delayed_Payment'].value_counts()

In [ ]:
# it might be an error in data entry.
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].abs()

In [ ]:
df[df['Num_of_Delayed_Payment'] < 0].shape[0]

#### Changed_Credit_Limit

In [ ]:
df[df['Changed_Credit_Limit'] == -6.450000].shape[0]

In [ ]:
df[df['Changed_Credit_Limit'] < 0]['Changed_Credit_Limit'].value_counts()

Negative values here could indicate adjustments or corrections where there is increase or decrease in the credit limit change.

In [ ]:
df.iloc[:, 0:15].describe()

In [ ]:
df.iloc[:, 15:].describe()

### Handle Missing Values

In [ ]:
df.isnull().sum()

In [ ]:
# We decided to drop columns with more than 20% missing values
missing_values = df.isnull().sum()
missing_percentage = {col: (missing / 50000) * 100 for col, missing in missing_values.items()}

missing_percentage_df = pd.DataFrame.from_dict(missing_percentage, orient='index', columns=['Missing_Percentage'])
missing_percentage_df.sort_values('Missing_Percentage')

In [ ]:
# Credit_Mix with 20.6% missing values
df.drop('Credit_Mix', axis=1, inplace=True)

No more columns to drop. Let's impute missing values instead.

As 'Customer ID' is crucial for identifying and linking individual records across the dataset, I decided to drop rows with missing 'Customer ID' to maintain data integrity and ensure consistency in analysis.

In [ ]:
df.dropna(subset=['Customer_ID'], inplace=True)

In [ ]:
df.isnull().sum()

It was a good choice, we've significantly reduced the number of missing values by only dropping rows with null values in Customer_ID.

#### Impute the missing values

In [ ]:
# Categorical Columns
df['Payment_Behaviour'] = df['Payment_Behaviour'].replace(np.nan, df['Payment_Behaviour'].mode()[0])
df['Type_of_Loan'] = df['Type_of_Loan'].replace(np.nan, df['Type_of_Loan'].mode()[0])
df['Occupation'] = df['Occupation'].replace(np.nan, df['Occupation'].mode()[0])

In [ ]:
numericalCols = df.select_dtypes(include='number').columns
numericalCols

In [ ]:
# To reduce the overhead of using KNN, we will not apply KNN imputation on all columns.
numCols_knn = ['Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Monthly_Balance']

In [ ]:
df.columns

In [ ]:
df['Num_of_Loan'] = df['Num_of_Loan'].replace(np.nan, 0)
for col in df.iloc[:, -9:]:
    df[col] = df[col].replace(np.nan, 0)

In [ ]:
imputer = SimpleImputer(strategy='median')
df['Age'] = imputer.fit_transform( df[['Age']] )
df['Credit_History_Age(months)'] = imputer.fit_transform( df[['Credit_History_Age(months)']] )
df['Credit_History_Age(years)'] = imputer.fit_transform( df[['Credit_History_Age(years)']] )
df['Num_Bank_Accounts'] = imputer.fit_transform( df[['Num_Bank_Accounts']] )

In [ ]:
for col in numCols_knn:
    imputer = KNNImputer()
    df[col] = imputer.fit_transform( df[[col]] )

In [ ]:
df.isnull().sum()

In [ ]:
df.iloc[:, :15].describe()

In [ ]:
df.iloc[:, 15:].describe()

##### Datatypes

In [ ]:
# some integer values we convert it to float erlier due to NaN values, return them back to integer.
df.info()

Age, Num_Bank_Accounts, Num_of_Loan, Num_of_Delayed_Payment, Num_Credit_Inquiries, Credit_History_Age(months), Has_Student_Loan,mHas_Not_Specified, Has_Personal_Loan, Has_Mortgage_Loan, Has_Payday_Loan, Has_Auto_Loan, Has_Credit_Builder_Loan, Has_Home_Equity_Loan, Has_Debt_Consolidation_Loan.

In [ ]:
cols_to_convert = ['Age', 'Num_Bank_Accounts', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Credit_History_Age(months)', 
                   'Has_Student_Loan', 'Has_Not_Specified', 'Has_Personal_Loan', 'Has_Mortgage_Loan', 'Has_Payday_Loan', 'Has_Auto_Loan', 
                   'Has_Credit_Builder_Loan', 'Has_Home_Equity_Loan', 'Has_Debt_Consolidation_Loan']

df[cols_to_convert] = df[cols_to_convert].astype(int)

In [ ]:
df.info()

### Handle Outliers

In [ ]:
numericCols = df.select_dtypes(include='number').columns
numericCols

In [ ]:
# show box plot before handing the outliers
for col in numericCols:
    fig = px.box(df, col, title=f'{col}')
    fig.show()

In [ ]:
for col in numericCols:
    outliers_indices = detect_outliers(df, 0, [col])
    df.loc[outliers_indices, col] = df[col].median()

In [ ]:
# show box plot after outliers handling
for col in numericCols:
    fig = px.box(df, col, title=f'{col}')
    fig.show()

#### Extract Age_Category from Age Column

In [ ]:
def extractAge(age):
    if age < 20:
        return 'Teens'
    elif 20 <= age < 40:
        return 'Adults'
    else:
        return 'Mature Adults'

In [ ]:
df['Age_Category'] = df['Age'].apply(extractAge)

In [ ]:
df[['Age', 'Age_Category']].sample(50)

### Statistics of Categorical Columns

In [ ]:
df.describe(include='O')

## Save to a CSV File

In [ ]:
df.to_csv('Bank Data (cleaned).csv', index=False)

## Data Analysis and Visualization

In [ ]:
df = pd.read_csv('Bank Data (cleaned).csv')

In [ ]:
# remember that Customer_ID is repeated every month. In some cases, we need customer's data only once.
df_unique = df[df['Customer_ID'].duplicated() == False]
df_unique.head()

In [ ]:
# for loan types, to apply some visualizations.. we need to melt the columns of types. 
loan_columns = df.iloc[:, -10:-1]
df_melt = df_unique.melt(id_vars=['Customer_ID', 'Num_of_Loan', 'Age', 'Total_EMI_per_month'], value_vars=loan_columns, var_name='Loan_Type', value_name='Type_Count')

#### Top 10 Occupations with the Highest Annual Income

In [ ]:
df_unique.groupby('Occupation')['Annual_Income'].mean().sort_values(ascending=False).reset_index().head(10)

We used unique_df instead of df because in `df`, some customers appear twice, others appear four times, etc. As a result, the weights are not equal.

#### Month with higher Num_Credit_Inquiries

In [ ]:
df.groupby('Month')['Num_Credit_Inquiries'].mean().idxmax()

In [ ]:
df.groupby('Month')['Num_Credit_Inquiries'].mean().head(1)

December has higher number of credit inqueries with average 7-8 inqueries.

#### Most Common Loan Type

In [ ]:
df_melt.groupby('Loan_Type')['Type_Count'].sum().idxmax()

#### Top 10 Customers with the Highest Monthly Balance

In [ ]:
df.groupby('Customer_ID')['Monthly_Balance'].mean().sort_values(ascending=False).head(10)

#### Top 10 Customers with the Most Bank Accounts

In [ ]:
df_unique.sort_values('Num_Bank_Accounts', ascending=False)[['Customer_ID', 'Num_Bank_Accounts']].head(10)

#### Worst Customers in Terms of Delayed Payments

In [ ]:
max_num_dp = df['Num_of_Delayed_Payment'].max()
print(f'These customers have {max_num_dp} delayed payments:')

df[ df['Num_of_Delayed_Payment'] == max_num_dp]['Customer_ID']

#### Age Group Distribution

In [ ]:
px.histogram(df_unique, 'Age_Category', text_auto=True)

In [ ]:
px.pie(df_unique, 'Age_Category')

The majority of customers are in the Adults age group. In contrast, Teens are less than 10% of the customers.

#### Age distribution of customers

In [ ]:
px.histogram( df , 'Age', nbins = 20)

The peak is around 30-34, indicating a higher concentration of customers in that age range.

In [ ]:
ff.create_distplot([df['Age']], ['Age'], show_hist=False, show_rug=False)

#### Most common occupations among customers

In [ ]:
px.histogram( df.drop_duplicates(subset='Customer_ID') , 'Occupation')

Lawyer is the most common occupation.
It’s important to highlight that this maximum count includes imputed values for missing occupation data.
However, since we imputed categorical data using the mode, the reported value is still correct.

In [ ]:
df['Occupation'].value_counts().idxmax()

#### Distribution of Payment Behavior

In [ ]:
px.histogram(df, 'Payment_Behaviour')

Most common payment behaviour is Low-spent small-value payment.

and less common               is Low-spent large-value payment.

In [ ]:
df['Payment_Behaviour'].value_counts()

##### Payment Behaviour with Payment of Min Account vs Intrest Rate 

In [ ]:
temp = df.groupby(['Payment_Behaviour', 'Payment_of_Min_Amount']).sum().reset_index(level=[0,1])
px.bar(temp, 'Payment_Behaviour', 'Interest_Rate', 'Payment_of_Min_Amount', barmode='group')

Most of those who has higher interest rate and make Low-spent small-value payment have already paid for the minimum payment.

##### Payment Behaviour correspond to Toatal EMI per Month

In [ ]:
temp = df.groupby('Payment_Behaviour').sum().reset_index(level=[0])
px.bar(temp, 'Payment_Behaviour', 'Total_EMI_per_month')

Customers who make Low-spent small-value payment tend to have higher EMI amounts.


#### Density of Credit Inquiries and Number of Loans

In [ ]:
px.violin(df, 'Num_of_Loan', 'Num_Credit_Inquiries')

It indicates that the distribution of credit inquiries is relatively consistent across different numbers of loans.

Higher loan numbers have less variability in the number of credit inquiries.

#### Credit History Age

In [ ]:
px.scatter(df, 'Credit_History_Age(months)', 'Credit_History_Age(years)')

It is normal because the year is extracted from the month.

In [ ]:
px.scatter(df, 'Credit_History_Age(years)', 'Age')

It's clear that there is no relation between age and credit history.

I thought as age increases, credit history increases.

#### Annual Income and Monthly Inhand Salary

In [ ]:
px.scatter(df, 'Annual_Income', 'Monthly_Inhand_Salary')

There is a positive correlation between Annual Income and Monthly Inhand Salary.

The data points are concentrated along an upward trend, suggesting a direct relationship between them.

#### Usage of Loan Types

In [ ]:
temp = df_melt.groupby(['Loan_Type']).sum().reset_index(level=[0])
px.bar(temp, 'Loan_Type', 'Type_Count')

##### Most Common Loan Types

In [ ]:
temp.sort_values('Type_Count', ascending=False)['Loan_Type']

Payday loan is most commonly used type.

#### Credit History by years

In [ ]:
ff.create_distplot([df['Credit_History_Age(years)']], ['Credit_History_Age(years)'])

Most customers have a credit history of 18-19 years.

#### Delay From Due Date for Age Categories

In [ ]:
px.histogram(df, 'Delay_from_due_date', color = 'Age_Category', nbins = 50, barmode='group')

Most teens and adults are deayed by 18-19 days.

Most mature adults delayed by 12-13 days.

#### How does the annual income vary across different occupations?

In [ ]:
temp = df.groupby('Occupation').sum().reset_index(level=[0])
px.bar(temp, 'Occupation', 'Annual_Income')

Architect has higher Annual Income after the Lawyer.

Journalist has lower Annual Income.

#### How does the number of bank accounts vary with annual income?

In [ ]:
temp = df.groupby('Num_Bank_Accounts').sum().reset_index(level=[0])
px.bar(temp, 'Num_Bank_Accounts', 'Annual_Income')

#### Annual income and num of loans

In [ ]:
temp = df_unique.groupby('Num_of_Loan').sum().reset_index()
px.bar(temp, 'Num_of_Loan', 'Annual_Income')

customers with higher annual incomes more likely to have less loans.

#### Salary and num of loans

In [ ]:
temp = df_unique.groupby('Num_of_Loan').sum().reset_index()
px.bar(temp, 'Num_of_Loan', 'Monthly_Inhand_Salary')

customers with higher salary have less loans.

In [ ]:
df.columns

#### Distribution of monthly finantial behaviour columns

In [ ]:
ff.create_distplot([df['Monthly_Balance'], df['Total_EMI_per_month'], df['Amount_invested_monthly']], ['Monthly_Balance','Total_EMI_per_month','Amount_invested_monthly'], show_hist=False, show_rug=False)